# OSMOSIS Winter

This notebook runs [GOTM](https://gotm.net/) with initial conditions and surface forcing during the winter months (Sep. 22, 2012 - Dec. 5, 2012) of the Ocean Surface Mixing, Ocean Submesoscale Interaction Study in the northeast Atlantic (OSMOSIS, 48.7$^\circ$N, 16.2$^\circ$W; [Damerell et al., 2016](https://doi.org/10.1002/2015JC011423)). See Fig. 2 of [Li et al., 2019](https://doi.org/10.1029/2019MS001810).

In [1]:
import sys
import copy
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='OSMOSIS-Winter-KC04', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /home/qingli/work/gotm/A2022_SMCLT_in_GOTM/gotm/code
   gotmdir_data: /home/qingli/work/gotm/A2022_SMCLT_in_GOTM/gotm/data
  gotmdir_build: /home/qingli/work/gotm/A2022_SMCLT_in_GOTM/gotm/build
    gotmdir_exe: /home/qingli/work/gotm/A2022_SMCLT_in_GOTM/gotm/exe
    gotmdir_run: /home/qingli/work/gotm/A2022_SMCLT_in_GOTM/gotm/run
 gotmdir_figure: /home/qingli/work/gotm/A2022_SMCLT_in_GOTM/gotm/figure
   gotmdir_tool: /home/qingli/work/gotm/A2022_SMCLT_in_GOTM/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 11.3 ms, sys: 21 ms, total: 32.3 ms
Wall time: 91.2 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfg = m.init_config()

Generating default configuration at '/home/qingli/work/gotm/A2022_SMCLT_in_GOTM/gotm/run/OSMOSIS-Winter-KC04/gotm.yaml'...
Done!


 ------------------------------------------------------------------------
 GOTM started on 2022/06/26 at 17:14:40
 ------------------------------------------------------------------------
    initialize_gotm
 ------------------------------------------------------------------------
        Reading configuration from: gotm.yaml
        configuring modules ....
    init_airsea_yaml
        done
    init_observations_yaml
    init_stokes_drift_yaml
        done
    init_turbulence_yaml
        done.
    init_cvmix_yaml
        done.
    init_meanflow_yaml
        done
    init_eqstate_yaml
        done.
 Your configuration has been written to /home/qingli/work/gotm/A2022_SMCLT_in_GOTM/gotm/run/OSMOSIS-Winter-KC04/gotm.yaml.
STOP 0


Update the configuration

In [6]:
# setup
title = 'OSMOSIS - Winter'
nlev = 200
cfg['title'] = title
cfg['location']['name'] = 'OSMOSIS'
cfg['location']['latitude'] = 48.7
cfg['location']['longitude'] = -16.2
cfg['location']['depth'] = 200.0
cfg['time']['start'] = '2012-09-22 00:00:00'
cfg['time']['stop']  = '2012-12-05 03:00:00'
cfg['time']['dt']    = 60.0
cfg['grid']['nlev']  = nlev

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'hour'
cfg['output']['gotm_out']['time_step'] = 3
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = os.path.dirname(os.path.dirname(m.environ['gotmdir_data']))+'/OSMOSIS-Winter'
cfg['temperature']['method'] = 'file'
cfg['temperature']['file'] = datadir+'/t_prof.dat'
cfg['salinity']['method'] = 'file'
cfg['salinity']['file'] = datadir+'/s_prof.dat'
cfg['surface']['fluxes']['heat']['method'] = 'file'
cfg['surface']['fluxes']['heat']['file'] = datadir+'/heat_flux.dat'
cfg['surface']['fluxes']['tx']['method'] = 'file'
cfg['surface']['fluxes']['tx']['file'] = datadir+'/momentum_flux.dat'
cfg['surface']['fluxes']['tx']['column'] = 1
cfg['surface']['fluxes']['ty']['method'] = 'file'
cfg['surface']['fluxes']['ty']['file'] = datadir+'/momentum_flux.dat'
cfg['surface']['fluxes']['ty']['column'] = 2
cfg['surface']['swr']['method'] = 'file'
cfg['surface']['swr']['file'] = datadir+'/swr.dat'
cfg['surface']['precip']['method'] = 'file'
cfg['surface']['precip']['file'] = datadir+'/pme.dat'
cfg['waves']['stokes_drift']['us']['method'] = 'exponential'
cfg['waves']['stokes_drift']['vs']['method'] = 'exponential'
cfg['waves']['stokes_drift']['dusdz']['method'] = 'us'
cfg['waves']['stokes_drift']['dvsdz']['method'] = 'vs'
cfg['waves']['stokes_drift']['exponential']['us0']['method'] = 'file'
cfg['waves']['stokes_drift']['exponential']['us0']['file'] = datadir+'/stokes_drift.dat'
cfg['waves']['stokes_drift']['exponential']['us0']['column'] = 1
cfg['waves']['stokes_drift']['exponential']['vs0']['method'] = 'file'
cfg['waves']['stokes_drift']['exponential']['vs0']['file'] = datadir+'/stokes_drift.dat'
cfg['waves']['stokes_drift']['exponential']['vs0']['column'] = 2
cfg['waves']['stokes_drift']['exponential']['ds']['method'] = 'file'
cfg['waves']['stokes_drift']['exponential']['ds']['file'] = datadir+'/stokes_drift.dat'
cfg['waves']['stokes_drift']['exponential']['ds']['column'] = 3

# water type
cfg['light_extinction']['method'] = 'custom'
cfg['light_extinction']['A']['method'] = 'constant'
cfg['light_extinction']['A']['constant_value'] = 0.57
cfg['light_extinction']['g1']['method'] = 'constant'
cfg['light_extinction']['g1']['constant_value'] = 0.5
cfg['light_extinction']['g2']['method'] = 'constant'
cfg['light_extinction']['g2']['constant_value'] = 13.4

# EOS -- use linear
cfg['eq_state']['form'] = 'linear_custom'
cfg['eq_state']['linear']['T0'] = 10.0
cfg['eq_state']['linear']['S0'] = 35.0
cfg['eq_state']['linear']['dtr0'] = -0.17
cfg['eq_state']['linear']['dsr0'] = 0.78

# damping on velocity: relaxation to zero with a 5-day decay time
cfg['velocities']['relax']['tau'] = 432000.0

In [7]:
cfgs = []
labels = []

In [8]:
# configure KC04-MY
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'mellor_yamada'
cfg['turbulence']['len_scale_method'] = 'mellor_yamada'
cfg['turbulence']['scnd']['method'] =  'quasi_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'kantha_clayson'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.2
cfg['turbulence']['turb_param']['compute_kappa'] = 'true'
cfg['turbulence']['my']['e6'] = 4.0
cfgs.append(copy.deepcopy(cfg))
labels.append('KC04-MY')

In [9]:
# configure KC04-k-omega
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'quasi_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'kantha_clayson'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.2
cfg['turbulence']['turb_param']['compute_kappa'] = 'true'
cfg['turbulence']['generic']['gen_m'] = 0.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = -1.0 
cfg['turbulence']['generic']['cpsi1'] = 0.55
cfg['turbulence']['generic']['cpsi2'] = 0.833
cfg['turbulence']['generic']['cpsi3minus'] = 0.0
cfg['turbulence']['generic']['cpsi3plus'] = 0.0 
cfg['turbulence']['generic']['sig_kpsi'] = 2.0 
cfg['turbulence']['generic']['sig_psi'] = 2.0
cfg['turbulence']['generic']['gen_d'] = -1.087
cfg['turbulence']['generic']['gen_alpha'] = -4.97
cfg['turbulence']['generic']['gen_l'] = 0.09
cfg['turbulence']['generic']['cpsi4'] = 0.15
cfgs.append(copy.deepcopy(cfg))
labels.append('KC04-k-omega')

In [10]:
cfg['waves']['stokes_drift']['us']['method'] = 'off'
cfg['waves']['stokes_drift']['vs']['method'] = 'off'
cfg['waves']['stokes_drift']['dusdz']['method'] = 'off'
cfg['waves']['stokes_drift']['dvsdz']['method'] = 'off'

In [11]:
# configure KC04-MY without Stokes drift
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'mellor_yamada'
cfg['turbulence']['len_scale_method'] = 'mellor_yamada'
cfg['turbulence']['scnd']['method'] =  'quasi_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'kantha_clayson'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.2
cfg['turbulence']['turb_param']['compute_kappa'] = 'true'
cfg['turbulence']['my']['e6'] = 4.0
cfgs.append(copy.deepcopy(cfg))
labels.append('KC04-MY-NS')

In [12]:
# configure KC04-k-omega without Stokes drift
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'quasi_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'kantha_clayson'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.2
cfg['turbulence']['turb_param']['compute_kappa'] = 'true'
cfg['turbulence']['generic']['gen_m'] = 0.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = -1.0 
cfg['turbulence']['generic']['cpsi1'] = 0.55
cfg['turbulence']['generic']['cpsi2'] = 0.833
cfg['turbulence']['generic']['cpsi3minus'] = 0.0
cfg['turbulence']['generic']['cpsi3plus'] = 0.0 
cfg['turbulence']['generic']['sig_kpsi'] = 2.0 
cfg['turbulence']['generic']['sig_psi'] = 2.0
cfg['turbulence']['generic']['gen_d'] = -1.087
cfg['turbulence']['generic']['gen_alpha'] = -4.97
cfg['turbulence']['generic']['gen_l'] = 0.09
cfg['turbulence']['generic']['cpsi4'] = 0.15
cfgs.append(copy.deepcopy(cfg))
labels.append('KC04-k-omega-NS')

## Run the model

In [13]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

CPU times: user 19 ms, sys: 14.5 ms, total: 33.5 ms
Wall time: 15 s
